In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [18]:
metadata_file = pd.read_csv('../Remaining Clean Data/updated_metadata.csv')

In [19]:
metadata_file.columns

Index(['image_id', 'species_name', 'latitude', 'longitude', 'altitude',
       'altitude_dem', 'accuracy', 'state_name', 'exif_byte_order',
       'phone_make', 'phone_camera_model', 'encoding_process',
       'rear_main_camera_megapixels', 'rear_main_camera_aperture_size',
       'rear_camera_flash_available', 'original_image_size',
       'original_image_resolution', 'image_date_time', 'outlier',
       'elevation_level', 'leaf_status', 'time_of_day', 'month'],
      dtype='object')

In [20]:
# for i in metadata_file["species_name"].unique():
#     file = metadata_file[metadata_file["species_name"]==i]
#     elev_min = file["altitude_dem"].min()
#     elev_max = file["altitude_dem"].max()
#     print(i)
#     print("minimum elevation : ",elev_min,"meter")
#     print("maximum elevation : ",elev_max,"meter")
#     print("=================================")

In [22]:
# temp = metadata_file[metadata_file['outlier']==True]

In [23]:
# temp['species_name'].value_counts()

In [24]:
metadata_file = metadata_file[metadata_file['outlier']==False]

In [ ]:
 
metadata_file["stratify_col"] = (
    metadata_file["phone_camera_model"] + "_" +
    metadata_file["time_of_day"] + "_" +
    metadata_file["leaf_status"] + "_" +
    metadata_file["elevation_level"]
)

# Function to sample from each species group
def safe_sample(group, n=2800):
    # Get counts of each stratum
    stratify_counts = group["stratify_col"].value_counts()
    
    stratified_sample = []

    for stratify_val, count in stratify_counts.items():
        stratify_group = group[group["stratify_col"] == stratify_val]
        # Calculate how many to sample from this stratum
        stratified_sample_size = min(n // len(stratify_counts), len(stratify_group))
        stratified_sample.append(stratify_group.sample(n=stratified_sample_size, random_state=42, replace=False))

    # Concatenate all the stratified samples
    stratified_sample_df = pd.concat(stratified_sample)

    # If we have fewer than 3000 rows (due to small strata), top up from remaining data (no duplicates!)
    remaining_count = n - len(stratified_sample_df)
    if remaining_count > 0:
        already_selected_ids = stratified_sample_df["image_id"].tolist()
        remaining_pool = group[~group["image_id"].isin(already_selected_ids)]
        stratified_sample_df = pd.concat([
            stratified_sample_df,
            remaining_pool.sample(n=remaining_count, random_state=42, replace=False)
        ])

    return stratified_sample_df

 
stratified_selection = metadata_file.groupby("species_name", group_keys=False).apply(safe_sample, n=2800)

 
print(stratified_selection["species_name"].value_counts())

/tmp/ipykernel_18840/2259361475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_file["stratify_col"] = (


species_name
Aesculus indica             2800
Buchanania lanzan           2800
Cedrus deodara              2800
Eucalyptus globulus         2800
Madhuca longifolia          2800
Mangifera sylvatica         2800
Phyllanthus emblica         2800
Pinus roxburghii            2800
Quercus leucotrichophora    2800
Rhododendron arboreum       2800
Senegalia catechu           2800
Shorea robusta              2800
Taxus baccata               2800
Name: count, dtype: int64


/tmp/ipykernel_18840/2259361475.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_selection = metadata_file.groupby("species_name", group_keys=False).apply(safe_sample, n=2800)


In [37]:
stratified_selection['elevation_level'].value_counts()

elevation_level
level4    7604
level5    7453
level3    7232
level2    7147
level1    6964
Name: count, dtype: int64

In [38]:
stratified_selection.drop(columns=['stratify_col'],inplace=True)

In [39]:
stratified_selection.to_csv('../Remaining Clean Data/stratified_selection.csv',index=False)